# Generative Adversarial Networks (GANs)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

## Intuition

Imagine you have a set of data points $X_{1:N}:\{x_1, x_2, \dots, x_N\}$ sampled from a distribution $p(x)$, and we want to acquire more samples from $p(x)$

### Inverse sampling

We could potentially use [*Inverse Sampling*](https://en.wikipedia.org/wiki/Inverse_transform_sampling) method, where we first sample $\widetilde{u}$ from a uniform distribution $U[0, 1]$ and pass it to the inverse of the *CDF* of $p(x)$ to generate our new sample $\widetilde{x}$:

$\tilde{u} \sim U[0, 1] \, , \, \tilde{x} = CDF^{-1}_{p(x)}(\tilde{u})$

This method works only when $p(x)$ is quite starightforward and computing its inverse CDF is not difficult.
This however is not the case most of the times.

### Rejection Sampling

In case $p(x)$ is quite convoluted such that inverse sampling is not feasible, one could possibly use [*Rejection Sampling*](https://en.wikipedia.org/wiki/Rejection_sampling) (a.k.a. accept/reject sampling). In this approach, a proposal distribution $q(x)$ with analytically closed form. Then an envelope function $e(x) = \alpha q(x)$ is created such that $e(x)=\alpha q(x) > p(x) \,\,\, \forall x, \,\,\, \alpha \ge 1$.
Then we follow this procedure:

At each iteration:

1) take a sample $\widetilde{x}$ from $q(x)$ and $\widetilde{u}$ from $U[0, 1]$. (*Remember that $q(x)$ must have been chosen to be an analytically simple distribution function, so taking a sample from it should be easy, e.g. using Inverse Sampling*).

2) If $\widetilde{u} < \frac{p(x)}{e(x)}$, accept the sample $\widetilde{x}$, otherwise reject it.

3) Go back to step-1 to try another sample.
![Rejection Sampling](files/rejectionsamplingcriterion.png)
Figure taken from: https://theclevermachine.wordpress.com/2012/09/10/rejection-sampling/

If we are lucky/clever and our proposal $e(x)$ is very close to the actual distribution $p(x)$, then we almost always get $\frac{p(x)}{e(x)} \approx 1$, hence most of the samples are accepted by our criteria, and sampling will be done faster.

Another example, where the underlying distribution is very intricate and we provide a simple proposal and scale it such that it envelopes the entire of our actual distribution and start the accept/reject sampling process:
![caption](files/RejectionSampling2.jpg)
Figure taken from: http://ct-cps.blogspot.com/2011/04/rejection-sampling-algorithm.html

### Other Sampling Methods

For high-dimensional data, Rejection sampling is super inefficient as a lot of samples are rejected due to the curse of dimensionality. Therefore, people tend to use other methods such as Importance Sampling, MCMC, Gibbs sampling, etc.

### Unknown distribution $p(x)$

Let's take make our problem a bit more complicated by assuming that we don't know the underlying distribution $p(x)$ and all we know about it is a set of already withdrawn samples $X_{1:N}:\{x_1, x_2, \dots, x_N\}$ .(e.g. a number of paintings drawn by an artist from 18th century).

Previously when we knew $p(x)$, we started from a uniformly sampled variable $\widetilde{u}$ and used either Inverse sampling, Rejection sampling, MCMC, or other methods to get from $\widetilde{u}$ to $\widetilde{x}$. (For instance, $\widetilde{u} \xrightarrow{CDF^{-1}_{p(x)}} \widetilde{x}$).
In other words, our knowledge about $p(x)$ was used in order to find the mapping. 

However, when $p(x)$ is unknown, this information is missing and we have to fill the black-box between $\widetilde{u}$ and $\widetilde{x}$ with some other technique. The main idea behind GANs is to use a **Generator** neural network to model that mapping $\widetilde{u} \xrightarrow{G(\theta)} \widetilde{x}$.

### Training the Generator $G(\theta)$ (by training a discriminator $D(\phi)$)

The generator network needs to learn how to map the input uniform sample to the real samples. One way would be to define a reconstruction loss function for it: $L = \|\widetilde{x} - x\|^2$. This direct comparison of the distribution samples and generated samples is not very straightforward and also does not produce good new samples.

Instead, we assess our generated samples in another way, by introducing a neural network called **discriminator** $D(\phi)$. Discriminator is trained to differentiate between the fake/generated samples and the real samples. In other words, at each iteraion of GAN training, the samples generated by $G(\widetilde{u}; \theta)$ are examined by $D(\widetilde{x}; \phi)$. The discriminator is trained to properly separate fake data from real data (two-class supervised learning). Moreover, Generator is trained such that its generated samples get a positive prodeiction from the discriminator. As we continue with this training process, Discriminator gets better and better at distinguishing positive and negative classes, and Generator gets better and better at generaing real-looking samples. This battle continues until discriminator cannot discriminate anymore!

Mathematically speaking, the objective of discriminator is a binary cross-entropy over positive labels (real data: $x$) and negative labels (fake data: $G(\widetilde{u}; \theta)$):

\begin{equation} 
\max_D\Bigg\{E_{x\sim p(x)}\Big[\log\big(D(x; \phi\big)\Big] \, + \, E_{\widetilde{u}\sim U[0, 1]}\Big[\log\big(1 - D\big(G(\widetilde{u}; \theta); \phi\big)\big)\Big]\Bigg\}
\tag{I}
\end{equation} 

In this objective function, $\theta$ is fixed and we only update the parameters of $D$ (i.e. $\phi$).


At the same time the objective of generator is to maximise the probability of getting a positive prediction label from the discriminator, or in other words, minimizing the value of $1 - D\big(G(\widetilde{u}; \theta); \phi\big)$ (while discriminator parameters, $\phi$, are fixed:
\begin{equation} 
\min_G\Big\{E_{\widetilde{u}\sim U[0, 1]}\big[\log(1 - D\big(G(\widetilde{u}; \theta); \phi\big)\big]\Big\}
\end{equation} 

In order to make this objective function more stable (c.f. page 116 in your hand-written notebook), it is re-written as:
\begin{equation} 
\max_G\Big\{E_{\widetilde{u}\sim U[0, 1]}\big[\log( D\big(G(\widetilde{u}; \theta); \phi\big)\big]\Big\}
\tag{II}
\end{equation} 

Now, the training process will alternately perform gradient ascent (or gradient descent on negative of these two log-likelihoods (equations **I** and **II**)) to update the networks and thereby, generate real-looking samples as a result.

### Define the Generator and Discriminator

In [ ]:
class Generator(nn.Module):
    def __init__(self, latent_dim=5, output_dim=2, hidden_sizes=(10, 5)):
        super().__init__()
        layers = []
        layers.extend([nn.Linear(latent_dim, hidden_sizes[0]),
                       nn.BatchNorm1d(hidden_sizes[0]),
                       nn.ReLU6(inplace=True)])
        for i in range(1, len(hidden_sizes)):
            layers.append(nn.Linear(hidden_sizes[i - 1], hidden_sizes[i]))
            layers.append(nn.BatchNorm1d(hidden_sizes[i]))
            layers.append(nn.ReLU6(inplace=True))
        layers.append(nn.Linear(hidden_sizes[-1], output_dim))
        self.layers = nn.ModuleList(layers)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        out = torch.sigmoid(x)
        return out


class Discriminator(nn.Module):
    def __init__(self, input_dim=2, hidden_sizes=(10, 5), dropout_p=0.2):
        super().__init__()
        layers = []
        layers.extend([nn.Linear(input_dim, hidden_sizes[0]),
                       nn.LeakyReLU(0.2, inplace=True),
                       nn.Dropout(dropout_p)])
        for i in range(1, len(hidden_sizes)):
            layers.append(nn.Linear(hidden_sizes[i - 1], hidden_sizes[i]))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            layers.append(nn.Dropout(dropout_p))
        layers.append(nn.Linear(hidden_sizes[-1], 2))
        self.layers = nn.ModuleList(layers)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

### Underlying distribution:

In [ ]:
sizes = [1000, 1000, 1000]
mu0 = np.array([-13, 15])
cov0 = np.array([[6, 5], [5, 6]])
data0 = np.random.multivariate_normal(mu0, cov0, size=sizes[0])
mu1 = np.array([0, -3])
cov1 = np.array([[20, 0], [0, 1]])
data1 = np.random.multivariate_normal(mu1, cov1, size=sizes[1])
mu2 = np.array([10, 13])
cov2 = np.array([[3, 0], [0, 3]])
data2 = np.random.multivariate_normal(mu2, cov2, size=sizes[2])

selected_data = [data0, data1, data2]

In [ ]:
synthetic_data = np.vstack(selected_data)
labels = np.concatenate([i * np.ones(selected_data[i].shape[0])
                         for i in range(len(selected_data))]).astype(int)

### Visualise the real data

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.plot(synthetic_data[:, 0], synthetic_data[:, 1], 
        '.', markersize=10, color='k')

In [ ]:
class RealDataGenerator(Dataset):
    def __init__(self, numpy_data, maxim=None, minim=None):
        super().__init__()
        if minim is not None:
            numpy_data = numpy_data - minim
        if maxim is not None:
            numpy_data = numpy_data / (maxim - minim)
        self.data = numpy_data

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        item = self.data[idx, :]
        return item

In [ ]:
data_max = synthetic_data.max(axis=0, keepdims=1)
data_min = synthetic_data.min(axis=0, keepdims=1)
synthetic_data_normalised = (synthetic_data - data_min) / (data_max - data_min)
batch_size = 128
dataloader = DataLoader(RealDataGenerator(synthetic_data,
                                          data_max,
                                          data_min),
                        batch_size=batch_size, shuffle=True)

In [ ]:
len(dataloader)

### Visualise the data coming from the dataloader

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
for idx, batch in enumerate(dataloader):
    batch_numpy = batch.data
#     print(batch_numpy.shape)
    ax.plot(batch_numpy[:, 0], batch_numpy[:, 1], 
            '.', markersize=10, color='k')
plt.show()

### Train the models

At each iteration, the discriminator is trained first using two batches of data from real samples and fake samples.
Then the generator is trained by setting its target to be positive and updating its parameters to make the predictions of its generated samples to be close to this fake target.

In [ ]:
def train_gan(generator, discriminator, latent_dim, dataloader,
              optimizer_G, optimizer_D, device, n_epochs,
              synthetic_data_normalised):
    for epoch in range(n_epochs + 1):
#         print(f"\n epoch {epoch}:")
        for idx, batch in enumerate(dataloader):
            bs = batch.shape[0]
            real_data = batch.to(device)
            real_data_labels = torch.ones(bs, 1).to(device)

            generator.to(device)
            discriminator.to(device)

            # ------------------------------------
            # Training the Discriminator:
            # ------------------------------------
            # generator.requires_grad_(False)
            # discriminator.requires_grad_(True)

            # generate fake data:
            input_noise = torch.rand(bs, latent_dim).to(device)
            fake_data = generator(input_noise)
            fake_data_true_labels = torch.zeros(bs, 1).to(device)

            # Combine reak and fake data into one training batch
            data_combined = torch.cat([real_data.double(),
                                       fake_data.double()], dim=0)
            labels_combined = torch.cat([real_data_labels, fake_data_true_labels], dim=0)

            # Shuffle the real and fake data
            perm = torch.randperm(bs * 2)
            labels_combined = labels_combined[perm, :]
            data_combined = data_combined[perm, :]

            # Check the discriminator output and update its parameters
            optimizer_D.zero_grad()
            discriminator_output = discriminator(data_combined.float())
            loss = F.cross_entropy(discriminator_output, labels_combined.long().view(-1))
            loss.backward()
            optimizer_D.step()

            # ------------------------------------
            # Training the Generator:
            # ------------------------------------
            # G.requires_grad_(True)
            # D.requires_grad_(False)

            input_noise = torch.rand(bs, latent_dim).to(device)
            fake_data = generator(input_noise)
            fake_data_fake_labels = torch.ones(bs, 1).to(device)

            optimizer_G.zero_grad()
            discriminator_output = discriminator(fake_data.float())
            loss = F.cross_entropy(discriminator_output, fake_data_fake_labels.long().view(-1))
            loss.backward()
            optimizer_G.step()
        if epoch % 50 == 0:
            input_noise = torch.rand(synthetic_data_normalised.shape[0],
                                     latent_dim).to(device)
            fake_data = generator(input_noise)
            batch_numpy = fake_data.cpu().data
            fig, ax = plt.subplots(1, 2, figsize=(10, 5))
            ax[0].plot(synthetic_data_normalised[:, 0],
                       synthetic_data_normalised[:, 1],
                       '.', markersize=3, marker='*', color='b')
            ax[0].set_title('Real Data')
            ax[1].plot(batch_numpy[:, 0], batch_numpy[:, 1],
                       '.', markersize=3, marker='s', color='k')
            ax[1].set_title(f'Generated data at epoch {epoch}')
            plt.show()

In [ ]:
lr = 0.0002
n_epochs = 500
latent_dim = 5
output_dim = synthetic_data.shape[1]
hidden_sizes_gen = (100, 50, 50)
hidden_sizes_dis = (100, 50, 50)
dropout_p = 0.1
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

# Instantiate the Generator and Discriminator
generator = Generator(latent_dim, output_dim, hidden_sizes_gen)
discriminator = Discriminator(output_dim, hidden_sizes_dis, dropout_p)

optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))


train_gan(generator, discriminator, latent_dim, dataloader,
          optimizer_G, optimizer_D, device, n_epochs,
          synthetic_data_normalised)